In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

非常好的追问 👍，这两个问题其实直指了 LoRA 的两个“核心工程逻辑”：

1. **为什么要做缩放（Scaling）**
2. **为什么合并时是“加法”而不是别的操作**

我们来系统地解释。

---

## 🧩 一、为什么要做缩放（Scaling）

LoRA 在训练时定义的更新为：
[
\Delta W = \frac{\alpha}{r} B A
]

其中：

* (r)：LoRA 的秩（rank）
* (\alpha)：缩放系数（scaling factor）
* (B,A)：可训练的低秩矩阵

### ✅ 1.1 为什么要缩放？

因为 (r) 越小，(BA) 的输出幅度（variance）往往越小，训练初期信号会变弱。
如果不缩放，低秩更新可能太小，不足以对冻结的权重 (W_0) 产生有效的影响。

**举个例子：**
假设 (r=1)，那 (B\in\mathbb{R}^{d_{out}\times 1})，(A\in\mathbb{R}^{1\times d_{in}})，
即 ΔW 实际上是一个向量外积。
这种外积天然方差比原始 (W_0) 小得多。

于是 LoRA 引入一个缩放：
[
\text{scale} = \frac{\alpha}{r}
]
例如设 (r=8, \alpha=16)，则缩放倍数为 2。

这样做相当于放大低秩更新的影响，让它在训练初期能与冻结权重的量级匹配。

---

### ✅ 1.2 为什么是 (\alpha / r) 而不是 (\alpha)？

因为 rank (r) 越大，矩阵 (BA) 的输出方差也会随着 (r) 增大（相当于更多方向叠加）。
所以用 (\frac{1}{r}) 抵消这个规模效应，保证不同 rank 下更新幅度大致一致。

换句话说：

* (r) 增大 → 更新信号本身变强
* 除以 (r) → 维持整体幅度稳定
* 再乘上 (\alpha) → 控制更新强度的“全局超参数”

> 📌 **直观理解：**
> (\frac{\alpha}{r}) 让 LoRA 的更新在不同 rank 下具有相似的“能量尺度”，
> 保证可比较性和训练稳定性。

---

### ✅ 1.3 训练稳定性和初始化

LoRA 里通常初始化：
[
B \sim 0, \quad A \sim \mathcal{N}(0, \sigma^2)
]

* 这样初始 (\Delta W \approx 0)，即一开始完全等价于冻结模型；
* 经过训练，(A,B) 慢慢调整；
* 缩放系数确保梯度传播幅度合适，不至于过弱或过强。

---

## ⚙️ 二、为什么合并策略是“加法”（merge via addition）

LoRA 的最终参数更新形式是：
[
W = W_0 + \frac{\alpha}{r} B A
]

也就是说：

* 训练完之后，可以直接把 LoRA 的增量权重 **加到原模型权重上**；
* 这样推理时不需要额外矩阵乘法，效率与原模型完全一致。

---

### ✅ 2.1 理论原因：线性可叠加性

因为 LoRA 的本质就是在原有线性变换上**添加一个线性扰动项**。

假设原层：
[
y = W_0 x
]

LoRA 改成：
[
y = (W_0 + \Delta W)x = W_0 x + \Delta W x
]

也就是说 LoRA 对输出的修改是：
[
\text{Δoutput} = (\Delta W)x
]
——这与全量微调 (W_0' = W_0 + \Delta W) 完全等价。
所以我们可以把训练好的 (BA) 直接**加回原权重矩阵**，模型功能不变。

---

### ✅ 2.2 工程原因：推理高效且可切换

合并为加法的好处：

* **推理时零额外计算**（只用一个 (W)）；
* **可动态加载不同任务的 LoRA 模块**：

比如你有一个共享的 (W_0)，但多个任务 LoRA：
[
W = W_0 + \frac{\alpha}{r}B_1A_1 \quad (\text{任务 1})
]
[
W = W_0 + \frac{\alpha}{r}B_2A_2 \quad (\text{任务 2})
]

推理时，只需切换不同 (A,B) 模块即可，无需重新训练整个模型。
这就是所谓 **“Adapter Fusion” 或 “Hot-swapping LoRA”**。

---

### ✅ 2.3 为什么不用乘法？

乘法形式（例如 (W = W_0 \times (I + \Delta))）在梯度传播上会：

* 引入非线性；
* 破坏模型原本的平衡；
* 使初始化不再对应于原模型（因为 (I+\Delta\neq I)）。

而加法：

* 初始 ΔW=0 → 模型完全等价于原始权重；
* 梯度平滑、训练稳定；
* 理论上等价于线性化近似（类似于泰勒展开的一阶项）。

---

## 💡 三、形象比喻总结

| 类比      | 含义            | LoRA 对应                      |
| ------- | ------------- | ---------------------------- |
| 🎚️ 缩放  | 调整微调信号的“音量”   | 保持更新幅度可控（(\frac{\alpha}{r})) |
| ➕ 加法    | 在线性层上叠加一个“补丁” | 最终权重 (W_0 + \Delta W)        |
| 🔄 可合并性 | 模块式“补丁”可热切换   | 适配多任务推理                      |

---

## ✅ 总结一句话

> **缩放**：为了在不同 rank 下保持更新幅度稳定，使训练信号量级可控；
> **加法合并**：因为 LoRA 是在线性层上添加一个低秩补丁，等价于线性增量，既保证数学一致性又方便工程部署。

---

如果你愿意，我可以接着画一个完整的 **“LoRA 前向传播 + 缩放 + 合并推理” 的逻辑图（带shape）**，直观显示：

* 原权重路径（冻结）；
* LoRA 分支（低秩更新）；
* 合并后的一步推理路径。
  是否帮你画？


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


class LinearLoRALayer(nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        merge=False,
        rank=8,
        lora_alpha=16,
        dropout=0.1,
    ):
        super().__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.merge = merge
        self.rank = rank

        # linear weight 的 Shape 是 (out_features, in_features), 正确的做法是 xW^T
        self.linear = nn.Linear(in_features, out_features)
        # 这里非常的重要，这里是实现的小细节

        if rank > 0:
            # 这里是为了标记 lora_a 和 lora_b 是可训练的参数
            self.lora_a = nn.Parameter(torch.zeros(out_features, rank))
            # lora_a 需要初始化为 高斯分布
            # 在调用凯明初始化的时候注释里写的高斯分布，调用的却是均匀分布，而且参数a的值设置的是根号5，但a表示的是leaky relu的负斜率系数，一般是0.01这样的小值，不可能超过1
            nn.init.kaiming_normal_(self.lora_a, a=0.01)

            self.lora_b = nn.Parameter(torch.zeros(rank, in_features))
            self.scale = lora_alpha / rank # * self.scale用于缩放lora的输出

            # linear 需要设置为不可以训练
            self.linear.weight.requires_grad = False
            self.linear.bias.requires_grad = False

        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        # 如果采用 merge 进行推理，
        # 那么会把 lora_a 和 lora_b 两个小矩阵的参数直接放到 linear.weight 中
        if merge:
            self.merge_weight()

    def forward(self, X):
        # X shape is (batch, seq_len, in_feature)
        # lora_a 是 out_features * rank
        if self.rank > 0 and not self.merge:
            output = self.linear(X) + self.scale * (X @ (self.lora_a @ self.lora_b).T)
        elif self.rank > 0 and self.merge:
            output = self.linear(X)
        else:
            output = self.linear(X)

        return self.dropout(output)

    def merge_weight(
        self,
    ):
        if self.merge and self.rank > 0:
            self.linear.weight.data += self.scale * (self.lora_a @ self.lora_b)

    def unmerge_weight(
        self,
    ):
        if self.rank > 0:
            self.linear.weight.data -= self.scale * (self.lora_a @ self.lora_b)


# 写一段测试代码
# Test the LoRALinear layer
batch_size = 32
seq_len = 128
in_features = 768
out_features = 512
rank = 8
lora_alpha = 16
dropout = 0.1

# Create a test input
x = torch.randn(batch_size, seq_len, in_features)

# Test regular mode (no merge)
lora_layer = LinearLoRALayer(
    in_features=in_features,
    out_features=out_features,
    rank=rank,
    lora_alpha=lora_alpha,
    dropout=dropout,
    merge=False,
)

# Forward pass
output = lora_layer(x)
print(
    f"Output shape (no merge): {output.shape}"
)  # Should be [batch_size, seq_len, out_features]

# Test merged mode
lora_layer_merged = LinearLoRALayer(
    in_features=in_features,
    out_features=out_features,
    rank=rank,
    lora_alpha=lora_alpha,
    dropout=dropout,
    merge=True,
)

# Forward pass with merged weights
output_merged = lora_layer_merged(x)
print(
    f"Output shape (merged): {output_merged.shape}"
)  # Should be [batch_size, seq_len, out_features]

# Test weight merging/unmerging
lora_layer.merge_weight()
output_after_merge = lora_layer(x)
lora_layer.unmerge_weight()
output_after_unmerge = lora_layer(x)

print(
    "Max difference after merge/unmerge cycle:",
    torch.max(torch.abs(output - output_after_unmerge)).item(),
)

Output shape (no merge): torch.Size([32, 128, 512])
Output shape (merged): torch.Size([32, 128, 512])
Max difference after merge/unmerge cycle: 2.804100275039673
